# 1. Čištění a spojování dat
V první fázi budeme číst data z MMR a trochu je počistíme.

In [1]:
import pandas as pd
import numpy as np
import urllib.request
import os
%matplotlib inline

In [2]:
url = 'http://www.strukturalni-fondy.cz/getmedia/d10fe3b0-15f5-4cdb-b916-64be5b4c5c4d/Prehled_projektu_20160406u.xlsx?ext=.xlsx'
fn = 'vstupy/prehled.xlsx'
dvs = 'vystupy' # kam se budou ukladat vystupy

if not os.path.exists(fn):
    urllib.request.urlretrieve(url, fn)

In [3]:
df = pd.read_excel(fn)
df = df.loc[df.loc[:, 'Pořadí v rámci v projektu (filtr)'] == 1] # kazdy projekt ma vic radku

Celkem máme 75 tisíc projektů z období 2007-2013, ke každému 28 indikátorů, většina je celkem sebevysvětlující.

In [4]:
df.shape

(75137, 28)

In [5]:
df.columns

Index(['Číslo a název programu', 'Číslo prioritní osy',
       'Číslo oblasti podpory', 'Číslo projektu', 'Název projektu',
       'Popis projektu', 'Žadatel', 'IČ žadatele', 'Hosp.právní forma',
       'Hosp.právní forma - skupina', 'Stav projektu',
       'Datum podpisu smlouvy/rozhodnutí', 'Adresa žadatele',
       'Kraj žadatele - kód', 'Kraj žadatele - název', 'Obec žadatele - kód',
       'Obec žadatele - název', 'Celkové zdroje - Sml/Dod',
       'Rozhodnutí/Smlouva o poskytnutí dotace (celkové zdroje)',
       'Rozhodnutí/Smlouva o poskytnutí dotace (veřejné prostředky celkem)',
       'Rozhodnutí/Smlouva o poskytnutí dotace (EU zdroje)',
       'Proplacené prostředky příjemcům - vyúčtované (veřejné prostředky celkem)',
       'Proplacené prostředky příjemcům - vyúčtované (EU zdroje)',
       'Certifikované prostředky (veřejné prostředky celkem)',
       'Certifikované prostředky (EU zdroje)', 'Název NUTS',
       'Pořadí v rámci v projektu (filtr)', 'Počet míst realizace'],
  

In [6]:
# nazvy sloupcu, at se to lip filtruje
prfo = 'Hosp.právní forma'
prfos = 'Hosp.právní forma - skupina'
prog = 'Číslo a název programu'
nuts = 'Název NUTS'
ico = 'IČ žadatele'
zad = 'Žadatel'
adr = 'Adresa žadatele'
obko = 'Obec žadatele - kód'
stav = 'Stav projektu'
dat = 'Datum podpisu smlouvy/rozhodnutí'
popis = 'Popis projektu'
nazev = 'Název projektu'
celk = 'Celkové zdroje - Sml/Dod'
euz = 'Rozhodnutí/Smlouva o poskytnutí dotace (veřejné prostředky celkem)'
eup = 'Proplacené prostředky příjemcům - vyúčtované (veřejné prostředky celkem)'

In [7]:
df.loc[df.loc[:, 'Certifikované prostředky (EU zdroje)'].str.startswith('PL').replace(np.nan,False)].head(2).T

,115197,115256
Číslo a název programu,CZ.3.22 OP ČR - Polsko,CZ.3.22 OP ČR - Polsko
Číslo prioritní osy,22.3,22.3
Číslo oblasti podpory,22.3.3,22.3.3
Číslo projektu,CZ.3.22/3.3.01/10.01931,CZ.3.22/3.3.01/11.02533
Název projektu,ROZVOJ MOTORSPORTU V ERN,Mezinárodní turnaj mladých basketbalových tale...
Popis projektu,Z důvodu neustále zvyšování prodejů terénních ...,"Spolupráce polského Klubu Koszykówkim ""StarBol..."
Žadatel,JAC-racing,Basketbalový klub Kondoři Liberec
IČ žadatele,26576180,27017052
Hosp.právní forma,"Sdruž.(svaz,spolek,spol","Sdruž.(svaz,spolek,spol"
Hosp.právní forma - skupina,"Komory, profesní a zájmové sdružení","Komory, profesní a zájmové sdružení"


***V datasetu od MMR je několik metrik pro částku hrazenou z dotací. Jde o Rozhodnutí, Proplacené prostředky a Certifikované prostředky. Je tak možné měřit objem dotací více způsoby. V celém tomto dokumentu se měří pouze ono Rozhodnutí. Změnou proměnné `euz` výše tuto funkcionalitu změníte.***

První tři projekty vypadají takto

In [8]:
df.head(3).T

,0,10,26
Číslo a název programu,CZ.1.01 OP Doprava,CZ.1.01 OP Doprava,CZ.1.01 OP Doprava
Číslo prioritní osy,1.1,1.1,1.1
Číslo oblasti podpory,1.1.1,1.1.1,1.1.1
Číslo projektu,CZ.1.01/1.1.00/06.0019,CZ.1.01/1.1.00/07.0006,CZ.1.01/1.1.00/07.0015
Název projektu,Optimalizace trati Benešov u Prahy - Strančice,Optimalizace trati Horní Dvořiště st. hranice ...,"Rekonstrukce kolejí č.1 a 2 v km 224,391-226,0..."
Popis projektu,"Délka rekonstruovaného úseku je 23,95 km. Stáv...",Vzhledem k již proběhlé částečné optimalizaci ...,V rámci stavby se předpokládá řešit rekonstruk...
Žadatel,"Správa železniční dopravní cesty, státní organ...","Správa železniční dopravní cesty, státní organ...","Správa železniční dopravní cesty, státní organ..."
IČ žadatele,70994234,70994234,70994234
Hosp.právní forma,Správa žel.dopr.cesty,Správa žel.dopr.cesty,Správa žel.dopr.cesty
Hosp.právní forma - skupina,Stát a jeho instituce a organizace,Stát a jeho instituce a organizace,Stát a jeho instituce a organizace


## Očista

### Projekty ČR-Polsko od nečeských subjektů
Nejjednodušší způsob jak vyfiltrovat žádosti od nečeských subjektů, je podívat se na kraj žadatele, ten je u těchto subjektů nevyplněný. Jediný false positive je EIB, kterou musíme vyjmout.

In [9]:
msk = (df.loc[:, 'Kraj žadatele - kód'].isnull()) & (df.loc[:, ico] != 8*'0')
df.loc[msk, [prog, ico, nazev, zad]].head(4)

,Číslo a název programu,IČ žadatele,Název projektu,Žadatel
118403,CZ.3.22 OP ČR - Polsko,072181712,Posilování komunikační dostupnosti Cieszyna a ...,Powiat Cieszyński
118405,CZ.3.22 OP ČR - Polsko,531412250,Zvýšení přeshraniční dostupnosti Nysa - Rejviz,Województwo Opolskie
118407,CZ.3.22 OP ČR - Polsko,531412600,Navrácení přeshraniční funkce cesty Prudník-Rý...,Powiat Prudnicki
118409,CZ.3.22 OP ČR - Polsko,000532955,Obnova historických vazeb - zkvalitnění techni...,Gmina Krzanowice


In [10]:
df.loc[msk].shape

(1687, 28)

Jde tedy o celkem 19 projektů.

In [11]:
df = df.loc[~msk]

### Nemáme žadatele ani IČO

In [12]:
msk = df.loc[:, zad].isnull() & df.loc[:, ico].isnull()
df.loc[msk].head(3).iloc[:, :8]

,Číslo a název programu,Číslo prioritní osy,Číslo oblasti podpory,Číslo projektu,Název projektu,Popis projektu,Žadatel,IČ žadatele
1487,CZ.1.01 OP Doprava,1.6,1.6.3,CZ.1.01/6.3.00/14.0392,Výměna dnové obšívky plavidla MS BEATRICE,Realizace projektu je nutná z důvodu havarijní...,NaN,NaN
3007,CZ.1.02 OP Životní prostředí,2.2,2.2.1,CZ.1.02/2.1.00/09.03845,Rekonstrukce vytápění hvozdu sladovny Kounice,Projekt rekonstrukce vytápění hvozdu sladovny ...,NaN,NaN
3058,CZ.1.02 OP Životní prostředí,2.2,2.2.1,CZ.1.02/2.1.00/11.09845,Zateplení a výměna zdroje vytápění,Výměna starého neekologického zdroje vytápění ...,NaN,NaN


Většinou živnostníci.

In [13]:
df.loc[msk,euz].sum()/10**9

7.6426827432900009

Dohromady skoro 8 miliard.

*Nastavíme nějaké prázdné hodnoty*

In [14]:
df.loc[msk, zad] = 'Neidentifikován'
df.loc[msk, ico] = '9'*8

### Nesprávné IČO
Je spousta projektů, kde IČO není správné české IČO. U zhruba 150 případů IČO chybí, u některých jde o zahraniční IČO (většina projektů ze spolupráce Česko-Polsko).

In [15]:
df.loc[:, ico].isnull().sum()

1735

In [16]:
nims = df.loc[:, ico].apply(lambda x: len(str(x)) == 8)
len(df)-nims.sum()

1739

In [17]:
# df.loc[df.loc[:, ico].apply(lambda x: type(x) == float or str(x).isdigit()),:].shape
df.loc[df.loc[:, ico].apply(lambda x: type(x) == float or (str(x).isdigit() and len(str(x)) == 8)),:].shape

(73446, 28)

In [18]:
df.shape

(73450, 28)

**Takže smažeme všechny projekty, kde IČO nemá osm znaků, ale zároveň není prázdné (na ty se ještě podíváme).**

In [19]:
df = df.loc[~((~df.loc[:, ico].isnull()) & ~nims)]

In [20]:
df.shape

(73446, 28)

In [21]:
df.loc[:, ico].isnull().value_counts()

False    71711
True      1735
Name: IČ žadatele, dtype: int64

#### Dvě stejně jmenné firmy, jedna má IČO, jedna nemá

In [22]:
ddd=df.loc[:, [zad, ico, obko]]
dg = ddd.groupby(zad)
for gr in dg:
    uq = gr[1].loc[:, ico].unique()
    if len(uq) == 1: continue
    #if gr[0].lower().startswith('obec'): continue
    #if gr[0].lower().startswith('město'): continue
    if len(gr[1].loc[:, obko].unique()) > 1: continue # jsou z ruznych obci, preskocit
    
    if len(uq) == 2 and pd.Series(uq).isnull().sum() == 1:
        # print(gr[0], uq)
        ic = gr[1].loc[:, ico].dropna().unique()[0] # ICO tyhle firmy
        df.loc[df.loc[:, zad] == gr[0], ico] = ic

#### Co zbývá

In [23]:
df.loc[:, ico].isnull().sum()

269

### Ruční IČO

In [24]:
nl = df.loc[:, ico].isnull()
df.loc[nl, [zad, ico, nuts]].drop_duplicates().head(10)

,Žadatel,IČ žadatele,Název NUTS
21377,"Českomoravská záruční a rozvojová banka, a.s.",NaN,Praha 1
27914,Agentura pro podporu podnikání a investic Czec...,NaN,Praha 2
34907,Česká agentura na podporu obchodu/CzechTrade,NaN,Praha 2
42486,Zaměstnavatelský svaz důlního a naftového prům...,NaN,Kladno
56143,OBEC KOSTELEC U HOLEŠOVA,NaN,Kostelec u Holešova
56144,OBEC CHOMÝŽ,NaN,Chomýž
60817,OBEC RŮŽOVÁ,NaN,Růžová
60818,OBEC VELKÁ BUKOVINA,NaN,Velká Bukovina
60919,Obec Sobíšky,NaN,Sobíšky
60936,OBEC HRACHOLUSKY,NaN,Hracholusky


Ručně jsem dohledal asi 150 IČO kódů

In [25]:
ruc = pd.read_csv('vstupy/rucni-ico.csv', dtype=object)
ruc.head(3)

,Žadatel,IČ žadatele,Název NUTS
0,"Českomoravská záruční a rozvojová banka, a.s.",44848943,Praha 1
1,Česká agentura na podporu obchodu/CzechTrade,00001171,Praha 2
2,OBEC KOSTELEC U HOLEŠOVA,00287342,Kostelec u Holešova


Tak je uložíme zpět

In [26]:
for el in ruc.iterrows():
    zz = el[1].loc['Žadatel']
    ii = el[1].loc['IČ žadatele']
    df.loc[(df.loc[:, zad] == zz) & (df.loc[:, ico].isnull()), ico] = ii

Zbylo toho fakt málo

In [27]:
df.loc[df.loc[:, ico].isnull(), zad].unique()

array([], dtype=object)

In [28]:
df = df.loc[~df.loc[:, ico].isnull()]

O tyto projekty náš dataset očistíme.

### Kofinancování

In [29]:
kofi = 'poměr vlastních financí (0-1)'
df.loc[:, kofi] = np.nan
fr = 1-df.loc[:, euz] / df.loc[:, celk]

fr = fr[fr >= 0] # mám pocit že u těch českopolských projektů je celková cena v eurech, ale dotace v korunách (a zlotých)
df.loc[fr.index, kofi] = fr.values

## Zmenšení datasetu

Jako první vezmeme z našeho seznamu projektů jen menší počet sloupců, nebudeme potřebovat všechny.

In [30]:
dfs = df.loc[:, [prog, ico, dat, nazev, popis, celk, euz, eup]]
dfs.columns = 'Program, IČO, Datum, Název, Popis, Celkové zdroje, Smlouva, Proplaceno'.split(', ')

In [31]:
dfs.head(3)

,Program,IČO,Datum,Název,Popis,Celkové zdroje,Smlouva,Proplaceno
0,CZ.1.01 OP Doprava,70994234,2008-07-02,Optimalizace trati Benešov u Prahy - Strančice,"Délka rekonstruovaného úseku je 23,95 km. Stáv...",4.041118e+09,2.600257e+09,2.566815e+09
10,CZ.1.01 OP Doprava,70994234,2008-07-02,Optimalizace trati Horní Dvořiště st. hranice ...,Vzhledem k již proběhlé částečné optimalizaci ...,1.282061e+09,8.415250e+08,8.739725e+08
26,CZ.1.01 OP Doprava,70994234,2008-06-11,"Rekonstrukce kolejí č.1 a 2 v km 224,391-226,0...",V rámci stavby se předpokládá řešit rekonstruk...,9.085452e+07,7.336503e+07,6.491240e+07


## Napojení dat z ARES
Náš seznam projektů můžeme zkombinovat s daty z RES (Registr ekonomických subjektů, MFČR)

In [32]:
res = pd.read_csv('vstupy/ares/subjekty.csv', dtype=object) # dtype=object, aby to nesmazalo nuly v IČO

In [33]:
# vezmeme jen par sloupcu
cln = 'ICO, OF, KPF, NPF, Nazev_NUTS4, Esa2010, Esa2010t, KPP, NACE, Nazev_NACE'.split(', ')

In [34]:
res = res.loc[:, cln]
res.columns = 'IČO, Žadatel, Právní forma (kód), Právní forma, NUTS4, ESA (kód), ESA, Zaměstnanců, NACE (kód), NACE'.split(', ')

In [35]:
res.head(3).T

,0,1,2
IČO,00000205,00001171,00001350
Žadatel,"Vojenské lesy a statky ČR, s.p.",Česká agentura na podporu obchodu,"Československá obchodní banka, a. s."
Právní forma (kód),301,331,121
Právní forma,Státní podnik,Příspěvková organizace,Akciová společnost
NUTS4,Praha,Praha,Praha
ESA (kód),11001,13110,12203
ESA,Nefinanční podniky veřejné,Ústřední vládní instituce,Instituce přijímající vklady kromě centrální b...
Zaměstnanců,2500 - 2999 zaměstnanců,100 - 199 zaměstnanců,5000 - 9999 zaměstnanců
NACE (kód),02200,70220,64190
NACE,Těžba dřeva,Ostatní poradenství v oblasti podnikání a řízení,Ostatní peněžní zprostředkování


In [36]:
dfs.shape

(73446, 8)

A konečně můžeme oba datasety spojit.

In [37]:
dfs = pd.merge(dfs, res, on = 'IČO', how='outer', indicator=True)

In [38]:
dfs.loc[:, '_merge'].value_counts()

both          71054
left_only      2392
right_only        3
dtype: int64

`right_only` nás nezajímaj, to jsou redudantní řádky z RES databáze

In [39]:
dfs = dfs.loc[~(dfs.loc[:, '_merge'] == 'right_only')]

Z těch, co máme jen mezi projekty, by měla být jen EIB (00000000) a fyzické osoby (99999999)

In [40]:
dfs.loc[dfs.loc[:, '_merge'] == 'left_only', 'IČO'].unique()

array(['99999999', '00000000'], dtype=object)

In [41]:
dfs.drop('_merge', axis=1, inplace=True)

Pa-rá-da.

## Typy podniků
Trochu setřídíme typy podniků, podle ESA kódů. Dál pak ještě můžeme určit, zda jde o veřejnou instituci nebo ne.

In [42]:
# url = 'http://www.mfcr.cz/assets/cs/media/Rozp-ramce-EU-85-2011_2016_Seznam-verejnych-spolecnosti-v-CR.xlsx'
# fn = 'vstupy/verejne-instituce.xlsx'

# if not os.path.exists(fn):
#     urllib.request.urlretrieve(url, fn)
    
# url = 'http://www.mfcr.cz/assets/cs/media/Rozp-ramce-EU-85-2011_2016_Seznam-vladnich-instituci-v-CR.xls'
# fn = 'vstupy/vladni-instituce.xls'

# if not os.path.exists(fn):
#     urllib.request.urlretrieve(url, fn)

# ver = pd.read_excel('vstupy/verejne-instituce.xlsx', skiprows=4)
# ver = ver.iloc[:, 1:]
# ver.head(5)

# vlad = pd.read_excel('vstupy/vladni-instituce.xls', skiprows=4)
# vlad = vlad.iloc[:, 1:]
# vlad.head(5)
# vvico = list(ver.loc[:, 'IČO'].values) + list(vlad.loc[:, 'IČO'].values)
# vvico = [str(l).rjust(8, '0') for l in vvico]
# vvico = set(vvico)

In [43]:
dfs.columns

Index(['Program', 'IČO', 'Datum', 'Název', 'Popis', 'Celkové zdroje',
       'Smlouva', 'Proplaceno', 'Žadatel', 'Právní forma (kód)',
       'Právní forma', 'NUTS4', 'ESA (kód)', 'ESA', 'Zaměstnanců',
       'NACE (kód)', 'NACE'],
      dtype='object')

In [44]:
(dfs.groupby(['ESA (kód)', 'ESA']).sum()/10**9).round(2)

,,Celkové zdroje,Smlouva,Proplaceno
ESA (kód),ESA,,,
11001,Nefinanční podniky veřejné,78.90,36.90,34.32
11002,Nefinanční podniky soukromé národní,226.72,99.99,94.16
11003,Nefinanční podniky soukromé pod zahraniční kontrolou,61.96,24.44,22.41
12100,Centrální banka,0.00,0.00,0.00
12203,Instituce přijímající vklady kromě centrální banky pod zahraniční kontrolou,0.01,0.01,0.01
12402,Fondy kolektivního investování (kromě FPT) soukromé národní,0.27,0.10,0.10
12502,Ostatní finanční zprostředkovatelé (kromě IC a PF) soukromí národní,0.01,0.01,0.01
12503,Ostatní finanční zprostředkovatelé (kromě IC a PF) pod zahraniční kontrolou,0.00,0.00,0.00
12602,Pomocné finanční instituce soukromé národní,0.03,0.02,0.02


In [45]:
esak = 'ESA (kód)'

První problém je, že spousta podniků (asi 10 %) nemá ESA kód. Budeme ho muset dogenerovat. Primárně skrz právní formu.

In [46]:
dfs.loc[dfs.loc[:, 'IČO'] == 8*'9', esak] = '00001'
dfs.loc[dfs.loc[:, 'IČO'] == 8*'9', 'ESA'] = 'Fyzické osoby'

dfs.loc[dfs.loc[:, 'IČO'] == 8*'0', esak] = '00002'
dfs.loc[dfs.loc[:, 'IČO'] == 8*'0', 'ESA'] = 'Evropská investiční banka'

In [47]:
prf = 'Právní forma'
nesa = dfs.loc[:, esak].isnull()

In [48]:
# dfs.loc[dfs.loc[:, esak].isnull()]

In [49]:
dfs.loc[nesa, prf].value_counts()

Společnost s ručením omezeným                    15
Příspěvková organizace                            9
Akciová společnost                                7
Sdružení (svaz, spolek, společnost, klub aj.)     3
Církevní organizace                               2
Obecně prospěšná společnost                       1
Komora (s výjimkou profesních komor)              1
Name: Právní forma, dtype: int64

In [50]:
dfs.loc[nesa].sum()/10**6

Celkové zdroje    130.206249
Smlouva            98.204275
Proplaceno         87.349603
dtype: float64

Jde o 100 milionů, tak hodím všechny zbývající mezi 'Nezařazeno'.

In [51]:
dfs.loc[nesa, esak] = '00000'
dfs.loc[nesa, 'ESA'] = 'Nezařazeno'

In [52]:
dfs.loc[:, esak].isnull().sum()

0

Nyní můžeme na hrubší kategorie

In [53]:
kat = 'Kategorie'
dfs.loc[:, kat] = np.nan

In [54]:
dfs.loc[dfs.loc[:, esak].str.startswith('15'), kat] = 'Neziskovka'
dfs.loc[dfs.loc[:, esak].str.startswith('12'), kat] = 'Finanční sektor'
dfs.loc[dfs.loc[:, esak] == '12100', kat] = 'Ostatní (veřejné)' # centrální banka
dfs.loc[dfs.loc[:, esak].str.startswith('11'), kat] = 'Nefinanční podnik'
dfs.loc[dfs.loc[:, esak] == '11001', kat] = 'Nefinanční podnik (veřejný)'

dfs.loc[dfs.loc[:, esak] == '13110', kat] = 'Vládní instituce (ústřední)'
dfs.loc[dfs.loc[:, esak] == '13130', kat] = 'Vládní instituce (místní)'

# dfs.loc[dfs.loc[:, esak].str.startswith('13'), kat] = 'Vládní instituce'
dfs.loc[dfs.loc[:, esak].str.startswith('14'), kat] = 'Ostatní (soukromé)'

In [55]:
dfs.loc[dfs.loc[:, esak] == '00000', kat] = 'Nezařazeno'
dfs.loc[dfs.loc[:, esak] == '00001', kat] = 'Ostatní (soukromé)' # fyzicke osoby
dfs.loc[dfs.loc[:, esak] == '00002', kat] = 'Ostatní (veřejné)' # EIB

In [56]:
dfs.loc[:, kat].isnull().sum()

0

In [57]:
(dfs.groupby(kat).sum()/10**9).round(2)

,Celkové zdroje,Smlouva,Proplaceno
Kategorie,,,
Finanční sektor,0.35,0.15,0.15
Nefinanční podnik,288.68,124.43,116.56
Nefinanční podnik (veřejný),78.90,36.90,34.32
Nezařazeno,0.13,0.10,0.09
Neziskovka,32.77,24.65,22.36
Ostatní (soukromé),17.53,8.64,8.27
Ostatní (veřejné),0.51,0.43,0.43
Vládní instituce (místní),346.25,241.14,217.01
Vládní instituce (ústřední),438.11,310.72,265.32


Nyní konečně přidáme info o veřejných institucích.

In [58]:
ver = 'Veřejná instituce'
dfs.loc[:, ver] = np.nan

In [59]:
dfs.loc[dfs.loc[:, kat].isin('Vládní instituce (ústřední), Vládní instituce (místní), Ostatní (veřejné), Nefinanční podnik (veřejný)'.split(', ')), ver] = True
dfs.loc[dfs.loc[:, kat].isin('Finanční sektor, Nefinanční podnik, Neziskovka, Ostatní (soukromé)'.split(', ')), ver] = False

In [60]:
dfs.loc[dfs.loc[:, ver].isnull()].sum()/10**6 # zbylo 'Nezařazeno, tedy cca 100 milionů

Celkové zdroje    130.206249
Smlouva            98.204275
Proplaceno         87.349603
dtype: float64

In [61]:
(dfs.groupby([ver, kat]).sum()/10**9).round(1)

Celkové zdroje  Smlouva  \
Veřejná instituce Kategorie                                              
False             Finanční sektor                         0.4      0.2   
                  Nefinanční podnik                     288.7    124.4   
                  Neziskovka                             32.8     24.7   
                  Ostatní (soukromé)                     17.5      8.6   
True              Nefinanční podnik (veřejný)            78.9     36.9   
                  Ostatní (veřejné)                       0.5      0.4   
                  Vládní instituce (místní)             346.2    241.1   
                  Vládní instituce (ústřední)           438.1    310.7   

                                               Proplaceno  
Veřejná instituce Kategorie                                
False             Finanční sektor                     0.1  
                  Nefinanční podnik                 116.6  
                  Neziskovka                         22.4  
                  Ostatní (soukromé)                  8.3  
True              Nefinanční podnik (veřejný)        34.3  
                  Ostatní (veřejné)                   0.4  
                  Vládní instituce (místní)         217.0  
                  Vládní instituce (ústřední)       265.3

## Export dat
Máme hotovo, můžeme ukládat.

In [62]:
dfs.to_csv('vystupy/projekty.csv', index=False)
dfs.to_excel('vystupy/projekty.xlsx', index=False)

In [64]:
dfs.drop('Popis', axis=1).to_csv('vystupy/projekty-bez-popisu.csv', index=False)
dfs.drop('Popis', axis=1).to_excel('vystupy/projekty-bez-popisu.xlsx', index=False)